<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN_041121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [14]:
import numpy as np
import matplotlib.pyplot as plt

# Single perceptron

In [15]:
class Neuron:
    """A single neuron with an activation function: "linear" by default, "sigmoid".
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias).""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight"""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ):
        self.activ = activ

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

In [16]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
# neu.set_activ('sigmoid')
neu.run([1,1])

5.0

In [17]:
class Perceptron:
    """A single neuron with the sigmoid activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0."""

    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias).""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        return self.sigmoid(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight"""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

## AND gate

In [18]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


In [19]:
#proverka
# a=[]
# a.append(Perceptron(inputs=2))
# a[1].weights

## OR gate

In [20]:
neuron = Perceptron(inputs=2)
neuron.set_weights([10,10,-5]) #OR gate

print("Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [21]:
neuron = Perceptron(inputs=2)
neuron.set_weights([-10,-10,15]) #NAND gate

print("Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer perceptron

In [22]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons.
       Attributes:
          layers:  A python list with the number of elements (incl bias) per layer. Incl the input layer.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate."""

    def __init__(self, layers, bias = 1.0, eta = 0.5):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    percep=Neuron(inputs = self.layers[i-1], bias = self.bias) # 
                    self.network[i].append(percep) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def printWeights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].activ,self.network[i][j].weights)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Perceptron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs)
        MSE = sum( error ** 2) / self.layers[-1]
        # STEP 3: Calculate the output error terms
        self.d[-1] = outputs * (1 - outputs) * (error)
        # STEP 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k] += delta
        return MSE

In [23]:
#test code
mlp1 = MultiLayerNeuron(layers=[2,2,1])  #mlp
mlp1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mlp1.printWeights()
print("MLP:")
print ("0 0 = {0:.10f}".format(mlp1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mlp1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mlp1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mlp1.run([1,1])[0]))
print ("1 1 =",mlp1.run([1,1]))

print ()
print (mlp1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"


Layer 2 Neuron 0 linear [-10 -10  15]
Layer 2 Neuron 1 linear [ 15  15 -10]
Layer 3 Neuron 0 linear [ 10  10 -15]

MLP:
0 0 = 35.0000000000
0 1 = 85.0000000000
1 0 = 85.0000000000
1 1 = 135.0000000000
1 1 = [135.]

[-10 -10  15]


In [24]:
class MultiLayerPerceptron:     
    """A multilayer perceptron class that uses the Perceptron class above.
       Builds a list of perceptrons.
       Attributes:
          layers:  A python list with the number of elements (incl bias) per layer. Incl the input layer.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate."""

    def __init__(self, layers, bias = 1.0, eta = 0.5):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    percep=Perceptron(inputs = self.layers[i-1], bias = self.bias) # 
                    self.network[i].append(percep) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def printWeights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].weights)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Perceptron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs)
        MSE = sum( error ** 2) / self.layers[-1]
        # STEP 3: Calculate the output error terms
        self.d[-1] = outputs * (1 - outputs) * (error)
        # STEP 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k] += delta
        return MSE

## XOR gate=(OR+NAND)+AND

In [25]:
#test code
mlp1 = MultiLayerPerceptron(layers=[2,2,1])  #mlp
mlp1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mlp1.printWeights()
print("MLP:")
print ("0 0 = {0:.10f}".format(mlp1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mlp1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mlp1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mlp1.run([1,1])[0]))
print ("1 1 =",mlp1.run([1,1]))

print ()
print (mlp1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"


Layer 2 Neuron 0 [-10 -10  15]
Layer 2 Neuron 1 [ 15  15 -10]
Layer 3 Neuron 0 [ 10  10 -15]

MLP:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098
1 1 = [0.00715281]

[-10 -10  15]


In [26]:
mlp2 = MultiLayerPerceptron(layers=[2,2,1])
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(5000):
    MSE = 0.0
    MSE += mlp2.bp([0,0],[0])
    MSE += mlp2.bp([0,1],[1])
    MSE += mlp2.bp([1,0],[1])
    MSE += mlp2.bp([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mlp2.printWeights()
    
print("MLP:")
print ("0 0 = {0:.10f}".format(mlp2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mlp2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mlp2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mlp2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

0.2641516880115423
0.26082119290114153
0.2599024388912923
0.25916288630209916
0.25726277812681975
0.24169258579748676
0.17924143014205796
0.1492578783179129
0.1403811749434996
0.13666415410787608
0.13469975878070936
0.13350489613787142
0.13270843374425428
0.13214256498537758
0.13172126252754035
0.13139618264913291
0.13113819192735396
0.13092874278757616
0.13075549280812776
0.13060992420782713
0.13048597676436963
0.1303792257137964
0.1302863677159522
0.1302048885132302
0.13013284173808093

Layer 2 Neuron 0 [-4.22412303 -7.8401579   0.80873514]
Layer 2 Neuron 1 [ 3.16830546 -7.72355068 -2.29885891]
Layer 3 Neuron 0 [-5.86241087  5.22702926 -0.03095852]

MLP:
0 0 = 0.0263390193
0 1 = 0.4910253720
1 0 = 0.9696940234
1 1 = 0.4936190861
